In [ ]:
from src import ds_Generator
import tensorflow as tf
from util import TRAIN_SPECIES_DF,SPECIES_SEED ,NUMBER_OF_SPECIES, SAVING_PATH
import datetime
import os

In [ ]:
train_ds, val_ds = DS_Generator.DS_Generator().generate_species_data(TRAIN_SPECIES_DF,augment=1,batch_size=64,seed=SPECIES_SEED)

In [ ]:
Input = tf.keras.Input((224,224,3))
# Choose some keras application
base = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=Input,
    input_shape=None,
    pooling="max",
    classifier_activation='softmax'
)
head = tf.keras.layers.Dense(NUMBER_OF_SPECIES,activation="softmax",)(base.output)
model = tf.keras.Model(inputs=Input, outputs=head,name="SomeNiceName")
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss="categorical_crossentropy",metrics=["acc"])

In [ ]:
# create folders
if model.name not in os.listdir(SAVING_PATH):
    os.makedirs(SAVING_PATH+model.name)
    os.makedirs(SAVING_PATH+model.name+"/logs")
    os.makedirs(SAVING_PATH+model.name+"/saves")

# timestamp for logging
time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

log_dir = SAVING_PATH+model.name+"/logs/" +time_stamp

# callback or performance
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = SAVING_PATH +model.name+"/saves/" +time_stamp +"/cp-{epoch:04d}.ckpt"

# callback or weights saving
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True)

model.fit(
train_ds,
epochs=35,
validation_data=val_ds,
callbacks=[cp_callback,tensorboard_callback])